In [1]:
import glob
import pandas as pd

In [2]:
user_infos = pd.concat([pd.read_csv(fpath) for fpath in glob.glob('../data/raw/files/treino/*.csv')])
user_infos.head()

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new
0,f98d1132f60d46883ce49583257104d15ce723b3bbda21...,Non-Logged,3,"c8aab885-433d-4e46-8066-479f40ba7fb2, 68d2039c...","1657146417045, 1657146605778, 1657146698738","76, 38, 41","20380, 21184, 35438","50.3, 18.18, 16.46","2, 1, 1","1657146417045, 1657146605778, 1657146698738"
1,2c1080975e257ed630e26679edbe4d5c850c65f3e09f65...,Non-Logged,60,"3325b5a1-979a-4cb3-82b6-63905c9edbe8, fe856057...","1656684240278, 1656761266729, 1656761528085, 1...","7, 80, 2, 1, 7, 62, 26, 44, 4, 4, 14, 45, 13, ...","6049, 210489, 8672, 10000, 30000, 123007, 9965...","25.35, 45.66, 35.3, 28.05, 36.53, 47.57, 55.33...","1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1...","1656684240278, 1656761266729, 1656761528085, 1..."
2,0adffd7450d3b9840d8c6215f0569ad942e782fb19b805...,Logged,107,"04756569-593e-4133-a95a-83d35d43dbbd, 29b6b142...","1656678946256, 1656701076495, 1656701882565, 1...","0, 0, 0, 0, 0, 44, 0, 0, 2, 1, 0, 0, 0, 44, 0,...","311274, 140000, 32515, 157018, 118689, 159243,...","67.58, 47.22, 41.52, 63.09, 51.38, 65.11, 71.9...","1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1...","1656678946256, 1656701076495, 1656701882565, 1..."
3,c1e8d644329a78ea1f994292db624c57980b2886cfbc2d...,Non-Logged,56,"1f2b9c2f-a2d2-4192-b009-09065da8ec23, 04756569...","1658333312180, 1658404553818, 1658408449062, 1...","8, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 2, 0, 1, 1...","182696, 91925, 30000, 273655, 126409, 42980, 1...","58.26, 72.66, 22.57, 59.89, 40.36, 36.35, 14.7...","1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1...","1658333312180, 1658404553818, 1658408449062, 1..."
4,e777d1f31d4d955b63d60acc13df336d3903f52ab8f8f4...,Non-Logged,4,"bebdeb3e-1699-43e0-a1b8-989f5a6ab679, f4b484a7...","1658766608801, 1658766608801, 1660084035094, 1...","579, 579, 7, 2","801396, 801396, 10000, 10000","78.74, 78.74, 16.71, 9.34","7, 7, 1, 1","1658766608801, 1658766608801, 1660084035094, 1..."


In [3]:
user_historys = user_infos[['userId', 'history','timestampHistory','numberOfClicksHistory','timeOnPageHistory','scrollPercentageHistory','pageVisitsCountHistory','timestampHistory_new']]

user_historys = user_historys.set_index('userId').apply(lambda row: row.str.split(','), axis=1)
user_historys = user_historys.apply(pd.Series.explode).reset_index()
user_historys['numberOfClicksHistory'] = user_historys['numberOfClicksHistory'].astype(int)
user_historys['timeOnPageHistory'] = user_historys['timeOnPageHistory'].astype(int)
user_historys['scrollPercentageHistory'] = user_historys['scrollPercentageHistory'].astype(float)
user_historys['timestampHistory'] = pd.to_numeric(user_historys['timestampHistory'], errors='coerce').fillna(0).astype(int)
user_historys['pageVisitsCountHistory'] = pd.to_numeric(user_historys['pageVisitsCountHistory'], errors='coerce').fillna(0).astype(int)
user_historys['history'] = user_historys['history'].str.strip()


user_historys.head()


KeyboardInterrupt



In [ ]:
user_historys.query('userId == "f98d1132f60d46883ce49583257104d15ce723b3bbda2147c1e31ac76f0bf069"')

In [ ]:
news_item = pd.concat([pd.read_csv(fpath) for fpath in glob.glob('../data/raw/itens/itens/*.csv')])
news_item.head()

In [ ]:
news_item.query('history == "0001eba9-55f4-477f-9de3-dc93b8b95574"')

# Premeira tentativa de recomendação heurística

In [ ]:
from datetime import datetime
import pandas as pd
from scipy.sparse import csr_matrix
import numpy as np
from sklearn.preprocessing import MinMaxScaler

interaction_columns = [
    'numberOfClicksHistory',
    'pageVisitsCountHistory',
    'timeOnPageHistory',
    'scrollPercentageHistory',
    'interaction_score'
]



def calc_recency_score(issued: pd.Series):
    """
    Calcula o score de recência para uma série de timestamps.
    
    O score é inversamente proporcional ao tempo decorrido, 
    dando mais prioridade a notícias mais recentes.
    
    :param issued: Série com os timestamps das notícias.
    :return: Série com os scores de recência.
    """
    # Converter os timestamps para milissegundos

    now = parse_date_timestamp(datetime.timestamp(datetime.now()))

    # Calcular o score de recência, evitando divisão por zero
    return 1 / np.maximum(now - parse_date_timestamp(issued), 1e-6)


def parse_date_timestamp(date):
    # Converte para datetime se ainda for string
    if isinstance(date, str):
        date = pd.to_datetime(date, errors='coerce')
    elif isinstance(date, float):
        return (date * 1000)

    return int(datetime.timestamp(date) * 1000)


def history_by(userId: str):
    return user_historys.query('userId == @userId').set_index('history')

def user_by(userId: str):
    return user_infos.query('userId == @userId')

import numpy as np
from scipy.sparse import csr_matrix

def build_interaction_matrix(df, interaction_columns):
    # Converter IDs para categorias
    user_id_category = df['userId'].astype('category')
    history_id_category = df['history'].astype('category')

    # Obter os índices das categorias
    row = user_id_category.cat.codes
    col = history_id_category.cat.codes

    # Extrair os dados de interação
    interaction_data = df[interaction_columns].values

    # Contar o número de usuários, itens e métricas
    num_users = len(user_id_category.cat.categories)
    num_items = len(history_id_category.cat.categories)
    num_metrics = len(interaction_columns)

    print(f"Número de usuários: {num_users}")
    print(f"Número de itens: {num_items}")
    print(f"Número de métricas: {num_metrics}")

    # Garantir que os dados de interação estejam no formato correto
    assert interaction_data.shape == (len(row), num_metrics), \
        f"Formato inesperado: interaction_data.shape = {interaction_data.shape}"

    # Construir a matriz de interação corretamente
    interaction_matrix = csr_matrix(
        (interaction_data[:, 0], (row, col)),
        shape=(num_users, num_items)
    )

    return user_id_category, history_id_category, interaction_matrix


# Criando score de recencia e score de populariadade por noticia

In [ ]:
scaler = MinMaxScaler()


count_visits_by_news = (user_historys
                        .groupby('history')['pageVisitsCountHistory']
                        .count())
news_item['issued'] = pd.to_datetime(news_item['issued'], errors='coerce')
news_item['recency_score'] = scaler.fit_transform(
    news_item['issued'].apply(calc_recency_score).fillna(0).values.reshape(-1, 1)
)


news_item = (news_item.set_index('page').join(count_visits_by_news, how='inner').reset_index())
news_item.rename(columns={'pageVisitsCountHistory': 'popularity_score'}, inplace=True)
news_item.rename(columns={'index': 'history'},inplace=True)
news_item.head()

# Preparando dados para treinamento do modelo

In [ ]:
news_item.shape

In [ ]:
import pandas as pd

# Obtém apenas os usuários logados para recomendação
users_info_logged = user_infos.query("userType == 'Logged'")[['userId']].set_index('userId')

# 🔗 Juntar os dados do histórico com as características dos itens
user_item = user_historys.join(users_info_logged, on='userId', how='inner')


# 
# # 🎯 Cálculo do interaction_score utilizando apenas as colunas disponíveis
user_item['interaction_score'] = (
        user_item['numberOfClicksHistory'] * 0.4 +
        user_item['pageVisitsCountHistory'] * 0.3 +
        user_item['timeOnPageHistory'] * 0.2 +
        user_item['scrollPercentageHistory'] * 0.1
)


# Treinando o modelo

In [ ]:
from sklearn.decomposition import TruncatedSVD

#em vez de usar pivot_table foi notado um alto custo de desempenho por isso ele foi substituido pelo csr_matrix
# # 🧱 Construção da matriz de interação
# interaction_matrix = merged.pivot_table(
#     index='userId',
#     columns='history',
#     values='interaction_score',
#     fill_value=0
# )

user_id_category, history_id_category,interaction_matrix =  build_interaction_matrix(user_item,interaction_columns)

# 📉 Aplicação do SVD
svd = TruncatedSVD(n_components=4)
user_factors = svd.fit_transform(interaction_matrix)
item_factors = svd.components_.T

In [ ]:
from sklearn.preprocessing import normalize

# Normalizar os vetores
user_vector = normalize(user_factors)
item_factors = normalize(item_factors)

# Criando a função de recomendação

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.preprocessing import normalize

def recomender(user_id, historySize, user_factors, item_factors, user_id_category,
               history_id_category, news_item, history_size_min_limit=50, top_k=5, top_p=None):

    if historySize > history_size_min_limit:

        # Se top_p for None, usar todos os itens
        top_p = min(top_p or len(item_factors), len(item_factors))

        try:
            # 🆔 Obter o índice numérico do usuário
            user_index = user_id_category.cat.categories.get_loc(user_id)

            # 🧠 Obter o vetor latente do usuário
            user_vector = user_factors[user_index].reshape(1, -1)


            # 🧮 Calcular a similaridade com os top_p itens
            similarities = cosine_similarity(user_vector, item_factors).flatten()

            # 🔝 Selecionar os índices dos itens mais similares dentro do top_p
            recommended_indices = similarities.argsort()[::-1][:top_p]

            # 🎯 Ajustar os índices para o mapeamento correto nos itens originais
            recommended_items = history_id_category.cat.categories[recommended_indices]
            print(user_id_category.cat.categories[:5])  # IDs de usuários
            print(history_id_category.cat.categories[:5])  # IDs de histórico
        except KeyError:
            print(f"User ID {user_id} não encontrado! O usuário pode estar offline")
            return []

        # 📊 Retornar os itens recomendados, agora incluindo a similaridade
        return (
            news_item.set_index('history')
            .loc[recommended_items]
            .assign(similarity=similarities[recommended_indices])
            .sort_values(['similarity', 'recency_score', 'popularity_score'], ascending=[False, False, False])
            .head(top_k)
        )

    else:
        # 🔙 Retorno padrão baseado em popularidade e recência
        return (
            news_item.set_index('history')
            .sort_values(['popularity_score', 'recency_score'], ascending=[False, False])
            .head(top_k)
        )


In [ ]:

(recomender(
    user_id='3f3491a8fc9ed10caad74f95d22efcff9537bcaa631e6ab4278280fdc1e7b9ad',
    historySize=60,
    top_k=74000,
    user_factors=user_vector,
    item_factors=item_factors,
    user_id_category=user_id_category,
    history_id_category=history_id_category,
    news_item=news_item
).reset_index().query("index in['e67c8cdf-3c55-4399-a864-3c1591225296','68ed45c8-71b7-4b88-bcde-1695d741aa42','6f81f339-dc75-4cb1-b12f-d20a344ae64d']")
 )

# 749
# 1647
#628
#202
#1023

In [ ]:
user_infos.query("historySize >50 and userType=='Logged'").head(2)

In [ ]:
user_by('e2ea40933f782f036277b6e250eb26a9d5e30ac17069806d206bcd48bcd7fa59')

# Salvando dados no datawarehorse

In [ ]:
from sqlalchemy import create_engine

# 💾 Salvar no SQLite
engine = create_engine('sqlite:///../data/refined/datawarehouse.db', echo=False)

# Salvar os DataFrames no banco de dados
user_item.to_sql('merged_data', con=engine, if_exists='replace', index=False)
# news_item.to_sql('news_item', con=engine, if_exists='replace', index=False)